In [1]:
import pandas as pd

In [2]:
base_path = 'pvqa/'

pvqa_dic_path = base_path + 'pvqa_dictionary.pkl'
test_img_id2idx_path = base_path + 'test_img_id2idx.pkl'
train_img_id2idx_path = base_path + 'train_img_id2idx.pkl'
val_img_id2idx_path = base_path + 'val_img_id2idx.pkl'

pvqa_dic_pkl = pd.read_pickle(pvqa_dic_path)
test_img_id2idx_pkl = pd.read_pickle(test_img_id2idx_path)
train_img_id2idx_pkl = pd.read_pickle(train_img_id2idx_path)
val_img_id2idx_pkl = pd.read_pickle(val_img_id2idx_path)

In [39]:
qas_base_path = base_path + 'qas/'

q2a_path = qas_base_path + 'q2a.pkl'
qid2a_path = qas_base_path + 'qid2a.pkl'
qid2q_path = qas_base_path + 'qid2q.pkl'
test_vqa_path = qas_base_path + 'test_vqa.pkl'
train_vqa_path = qas_base_path + 'train_vqa.pkl'
val_vqa_path = qas_base_path + 'val_vqa.pkl'
train_qa_path = qas_base_path + 'train/train_qa.pkl'

q2a_pairs = pd.read_pickle(q2a_path)
qid2a_pairs = pd.read_pickle(qid2a_path)
qid2q_pairs = pd.read_pickle(qid2q_path)
test_vqa_pairs = pd.read_pickle(test_vqa_path)
train_vqa_pairs = pd.read_pickle(train_vqa_path)
train_qa_pairs = pd.read_pickle(train_qa_path)
val_vqa_pairs = pd.read_pickle(val_vqa_path)

In [31]:
train_vqa_df = pd.DataFrame(train_vqa_pairs)
val_vqa_df = pd.DataFrame(val_vqa_pairs)
test_vqa_df = pd.DataFrame(test_vqa_pairs)

In [36]:
test_vqa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6761 entries, 0 to 6760
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   answer_type    6761 non-null   object
 1   img_id         6761 non-null   object
 2   label          6761 non-null   object
 3   question_id    6761 non-null   int64 
 4   question_type  6761 non-null   object
 5   sent           6761 non-null   object
dtypes: int64(1), object(5)
memory usage: 317.0+ KB


In [38]:
val_vqa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6279 entries, 0 to 6278
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   answer_type    6279 non-null   object
 1   img_id         6279 non-null   object
 2   label          6279 non-null   object
 3   question_id    6279 non-null   int64 
 4   question_type  6279 non-null   object
 5   sent           6279 non-null   object
dtypes: int64(1), object(5)
memory usage: 294.5+ KB


In [37]:
train_vqa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19755 entries, 0 to 19754
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   answer_type    19755 non-null  object
 1   img_id         19755 non-null  object
 2   label          19755 non-null  object
 3   question_id    19755 non-null  int64 
 4   question_type  19755 non-null  object
 5   sent           19755 non-null  object
dtypes: int64(1), object(5)
memory usage: 926.1+ KB


In [26]:
list(q2a_pairs.items())[:5] 

[('Where are liver stem cells (oval cells) located?',
  'in the canals of hering'),
 ('What are stained here with an immunohistochemical stain for cytokeratin 7?',
  'bile duct cells and canals of hering'),
 ('What are bile duct cells and canals of Hering stained here with for cytokeratin 7?',
  'an immunohistochemical stain'),
 ('Are bile duct cells and canals of Hering stained here with an immunohistochemical stain for cytokeratin 7?',
  'yes'),
 ('What shows dissolution of the tissue?', 'an infarct in the brain')]

In [27]:
list(qid2q_pairs.items())[:5]

[(100422000, 'Where are liver stem cells (oval cells) located?'),
 (100422001,
  'What are stained here with an immunohistochemical stain for cytokeratin 7?'),
 (100422002,
  'What are bile duct cells and canals of Hering stained here with for cytokeratin 7?'),
 (100422003,
  'Are bile duct cells and canals of Hering stained here with an immunohistochemical stain for cytokeratin 7?'),
 (100986000, 'What shows dissolution of the tissue?')]

In [41]:
train_qa_df = pd.DataFrame(train_qa_pairs)

In [43]:
train_qa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19755 entries, 0 to 19754
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     19755 non-null  object
 1   question  19755 non-null  object
 2   answer    19755 non-null  object
dtypes: object(3)
memory usage: 463.1+ KB
